In [ ]:
# 04_chaos_metrics.ipynb

from src.preprocessing import load_and_filter_strain
from src.emd_utils import compute_imfs
from src.chaos_metrics import compute_lle, compute_boxcount_fd

file_path = 'data/H-H1_LOSC_CLN_16_V1-1187007040-2048.hdf5'
strain_filtered, time, fs = load_and_filter_strain(file_path)

start_time = 0.5
end_time = 1.5
start_idx = int(start_time * fs)
end_idx = int(end_time * fs)
segment = strain_filtered[start_idx:end_idx]
segment = (segment - np.mean(segment)) / np.std(segment)

# Get IMFs
imfs = compute_imfs(segment)

# Compute chaos metrics
lle_results = []
fd_results = []

for i, imf in enumerate(imfs):
    norm_imf = (imf - np.mean(imf)) / np.std(imf)
    print(f"→ IMF {i+1}")
    try:
        lle = compute_lle(norm_imf)
        fd = compute_boxcount_fd(norm_imf)
        lle_results.append(lle)
        fd_results.append(fd)
        print(f"  LLE: {lle:.5f}, FD: {fd:.5f}")
    except Exception as e:
        print(f"  Error computing metrics: {e}")
        lle_results.append(np.nan)
        fd_results.append(np.nan)

# Save results for next notebook
import json
with open('results/chaos_metrics.json', 'w') as f:
    json.dump({'lle': lle_results, 'fd': fd_results}, f)